double check against <https://github.com/leelabcnbc/tang_jcompneuro/blob/master/results_ipynb/debug/gabor_debug/gabor_fitting_debug_complex_one_neuron.ipynb>

In [1]:
from tang_jcompneuro.io import load_image_dataset, load_neural_dataset

In [2]:
neuron_to_work_on = 33
y = load_neural_dataset('MkA_Shape')[:, neuron_to_work_on:neuron_to_work_on+1]
X = load_image_dataset('Shape_9500', trans=True, scale=0.5, subtract_mean=True, legacy_rescale=True)
print(X.min(), X.max(), X.mean())
X_mean = X.reshape(len(X), -1).mean(axis=1)
print(X_mean.shape, X_mean.min(), X_mean.max())

-0.85 0.983125 -7.86739094924e-19
(9500,) -1.7763568394e-16 2.48689957516e-16


In [3]:
datasets = (X, y, None, None, None, None)

In [4]:
# get trainer.
from tang_jcompneuro.model_fitting_gabor import get_trainer

In [5]:
trainer = get_trainer('complex')
results = trainer(datasets)

complex None
bad index 0/192


In [6]:
# pretty match what I got before last cell in previous notebook.
# difference might be due to CUDA version, PyTorch internal difference, etc.
results

{'attrs': {'corr_train': 0.45479408},
 'corr': 0.45479408,
 'model': functools.partial(<function save_model_to_hdf5_group at 0x2b42d4138598>, saved_params={'loc': array([[ 0.00583964],
        [-0.0955117 ]], dtype=float32), 'sigma': array([[ 0.19446135],
        [ 0.15373458]], dtype=float32), 'orientation': array([ 4.0409441], dtype=float32), 'frequency': array([ 2.22182369], dtype=float32), 'output_a': array([ 0.0057111], dtype=float32), 'output_b': -0.00022449138}),
 'y_test_hat': array([[ 0.02630009,  0.02630009,  0.02630009, ...,  0.10959955,
          0.10393838,  0.12891151]], dtype=float32)}

In [7]:
assert callable(results['model'])

In [8]:
# see if we can save models.
import h5py
from tempfile import NamedTemporaryFile
from tang_jcompneuro.gabor import load_model_from_hdf5_group

with NamedTemporaryFile() as f:
    with h5py.File(f.name) as f_tmp:
        results['model'](f_tmp)
        # probably I should not go out of this second with, as that may close f.
        # to avoid all these issues, best way would be create a temp dir, rather than a temp file.
        # but whatever.
        best_params_complex_OUT = load_model_from_hdf5_group(f_tmp)
print(best_params_complex_OUT)

{'frequency': array([ 2.22182369], dtype=float32), 'loc': array([[ 0.00583964],
       [-0.0955117 ]], dtype=float32), 'orientation': array([ 4.0409441], dtype=float32), 'output_a': array([ 0.0057111], dtype=float32), 'output_b': -0.00022449138, 'sigma': array([[ 0.19446135],
       [ 0.15373458]], dtype=float32)}
